# PyTorch CNN

## <a name="ackw"></a> Acknowledgements

The code in this notebook is edited from <a href="https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-in-pytorch/">Convolutional Neural Networks Tutorial in PyTorch</a>.

## <a name="overview"></a> Overview

In this notebook, we will use PyTorch to construct a CNN.

## <a name="ekf"></a> PyTorch CNN

In [1]:
import torch
import torch.nn as nn

 Our CNN will have two layers. The first layer consists of:

- ```nn.Conv2d```
- ```nn.ReLU```
- ```nn.MaxPool2d```

---

**Remark**

With CNNs, we often use ReLU layers. This layer will apply an element-wise  activation function over the input data thresholding, e.g. ```max(0,x)```, at zero giving us the same dimension output as the input to the layer.
Running this function over the input volume will change the pixel values but will not
change the spatial dimensions of the input data in the output. ReLU layers do not
have parameters nor additional hyperparameters.

---


The ```Conv2d``` is a convolution layer with 1 input channel and 32 output channels. The ```kernel_size``` argument is the size of the convolutional filter, $5\times5$ sized convolutional filters. If we wanted filters with different sized shapes in the $x$ and $y$ directions, w should supply a tuple $(x, y)$. Finally, we want to specify the padding argument. This takes a little bit more thought. The output size of any dimension from either a convolutional filtering or pooling operation can be calculated by the following equation:


$$W_{out} = \frac{W_{in} + F + 2P}{S} + 1$$

Where $W_{in}$ is the width of the input, $F$ is the filter size, $P$ is the padding and $S$ is the stride. The same formula applies to the height calculation, but seeing as our image and filtering are symmetrical the same formula applies to both. If we wish to keep our input and output dimensions the same, with a filter size of 5 and a stride of 1, it turns out from the above formula that we need a padding of 2. Therefore, the argument for padding in Conv2d is 2.

The next element in the sequence is a simple ReLU activation. The last element that is added in the sequential definition for self.layer1 is the max pooling operation. The first argument is the pooling size, which is 2 x 2 and hence the argument is 2. Second – we want to down-sample our data by reducing the effective image size by a factor of 2. To do this, using the formula above, we set the stride to 2 and the padding to zero. Therefore, the stride argument is equal to 2. The padding argument defaults to 0 if we don’t specify it – so that’s what is done in the code above. From these calculations, we now know that the output from self.layer1 will be 32 channels of 14 x 14 “images”.

kernel size stride 1 and padding 1. It is followed by the application of ```ReLU``` layer and then followed by the application of a $2\times 2$  ```MaxPool2d``` down sampling layer. This results as an $14\times14$ output serving as an input to the second layer.

After the two convolution layers, we apply a linear flatten operation which creates  $7 \times 7 \times 64 = 3164$ nodes, an intermediate layer of 1000 fully connected nodes and a ```softmax``` operation over the 10 output nodes to produce class probabilities. These layers represent the output classifier. 

The ```Cnn``` class defines our CNN. The class should override the ```forward``` function specifying how predictions should be made. This is done via the application of layers.

In [ ]:
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn, self).__init__()
        
        # first convolutional layer
        self.layer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        
        # second convolutional layer
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
    
    
    def train(self, x, criterion, optimizer):
        
        for i, (images, labels) in enumerate(x):
            
            # Run the forward pass
            outputs = self(images)
            loss = criterion(outputs, labels)
            loss_list.append(loss.item())

            # Backprop and perform Adam optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            acc_list.append(correct / total)

            if (i + 1) % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total) * 100))
        
        

## <a name="refs"></a> References